In [3]:
# %pip install backtesting pandas numpy matplotlib

In [ ]:
# %pip install

In [ ]:
import pandas as pd
from backtesting import Backtest, Strategy

# Load your data
data = pd.read_csv('your_data.csv', index_col='Date', parse_dates=True)

In [ ]:
class MyRealisticStrategy(Strategy):
    def init(self):
        # Load news events and swap rates
        self.news_events = pd.read_csv('news_events.csv', parse_dates=['Date'])
        self.news_events.set_index('Date', inplace=True)
        self.swap_rates = pd.read_csv('swap_rates.csv', parse_dates=['Date'])
        self.swap_rates.set_index('Date', inplace=True)

    def _open_position(self, signal, size, limit=None, stop=None, sl=None, tp=None):
        price = self.data.Close[-1]
        spread = 0.0002  # Adjust based on your typical spread
        if signal == 1:
            price += spread / 2
        else:
            price -= spread / 2
        return super()._open_position(signal, size, price, stop, sl, tp)

    def next(self):
        current_date = self.data.index[-1]

        # Check for news events
        if current_date in self.news_events.index:
            return

        # Check swap rates
        if current_date in self.swap_rates.index:
            swap_rate = self.swap_rates.loc[current_date, 'SwapRate']
            if swap_rate < 0:
                return

        # Your regular trading logic here

In [ ]:
bt = Backtest(data, MyRealisticStrategy, cash=10000, commission=.002)
stats = bt.run()